# Data exploration for city of Bremen

## Import required packages

In [1]:
import pandas as pd
import os
import folium
import geopandas as gpd
from shapely.ops import cascaded_union
from pandas.io.json import json_normalize
from folium.plugins import MarkerCluster

## Import raw data

In [2]:
raw_data = pd.read_csv('../data/raw/bremen.csv')
raw_data.drop(columns=['Unnamed: 0'], inplace=True) # Drop second index column
raw_data.head()

,p_spot,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,b_bike_type,p_name,p_number,p_lng,p_bike
0,True,0,2019-01-20 02:06:00,11281,first,4774295,5,50.808852,15,Biegenstraße/Cineplex,5155.0,8.773134,False
1,True,0,2019-01-20 14:16:00,11281,last,4774295,4,50.808852,15,Biegenstraße/Cineplex,5155.0,8.773134,False
2,True,0,2019-01-20 00:00:00,11169,first,4774543,5,50.795224,15,Südbahnhof,5173.0,8.763266,False
3,True,0,2019-01-20 01:55:00,11169,start,4774543,5,50.795224,15,Südbahnhof,5173.0,8.763266,False
4,True,0,2019-01-20 02:06:00,11169,end,4774368,4,50.804522,15,Frankfurter Straße/Psychologie,5159.0,8.770358,False


## Filter data for entrys of Bremen only
https://github.com/gboeing/urban-data-science/blob/master/19-Spatial-Analysis-and-Cartography/rtree-spatial-indexing.ipynb

### Import geodata of Bremen (PLZ based)

https://public.opendatasoft.com/explore/dataset/postleitzahlen-deutschland/table/?refine.note=Bremen

In [3]:
plz_df = gpd.read_file('../data/external/plz_bremen.geojson')

In [4]:
# Function that creates map with center such that all of Bremen is displayed.

def create_bremen_map(zoom = 11):
    
    m = folium.Map(location=[53.122962,8.7515937], zoom_start=zoom)
    
    return m

In [5]:
geo_data_bremen = create_bremen_map()

folium.Choropleth(plz_df).add_to(geo_data_bremen)

geo_data_bremen

### Get "boundaries" (min, max of latitude and longitude) of Bremen

In [6]:
boundaries = cascaded_union(plz_df.geometry).bounds
boundaries

(8.4815929, 53.0110367, 8.9907318, 53.22892479999999)

### Filter raw data such that we only keep data points within the boundaries of Bremen
- has to be done in preprocessor

#### Coarse filtering first for faster computation

In [7]:
bremen = raw_data[(raw_data['p_lat'] < boundaries[3]) &
         (raw_data['p_lat'] > boundaries[1]) &
         (raw_data['p_lng'] < boundaries[2])  &
         (raw_data['p_lng'] > boundaries[0])]

print(len(bremen))

1309128


## Total number of null values

- there are only null values (88) of the attribute p_number

In [8]:
bremen.isna().sum()

p_spot           0
p_place_type     0
datetime         0
b_number         0
trip             0
p_uid            0
p_bikes          0
p_lat            0
b_bike_type      0
p_name           0
p_number        88
p_lng            0
p_bike           0
dtype: int64

### Drop null values
- has to be done in preprocessor

In [9]:
bremen = bremen.dropna()

### Create geopandas data frame

In [10]:
bremen = gpd.GeoDataFrame(bremen, geometry=gpd.points_from_xy(bremen.p_lng.copy(), bremen.p_lat.copy()))

### Precise filtering for data points within Bremen

In [12]:
bremen = gpd.sjoin(bremen, plz_df[['geometry','plz']], how='left', op = 'within')

/Users/timockenga/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326'})
  "(%s != %s)" % (left_df.crs, right_df.crs)


#### Drop index of plz df and drop data points that are outside of Bremen borders

In [13]:
bremen = bremen.drop(columns=['index_right']).dropna()

## Reorder data frame
- has to be done in preprocessor

In [14]:
bremen = bremen[['datetime', 'b_number', 'p_spot', 'p_place_type',
                 'trip', 'p_uid', 'p_bikes', 'b_bike_type', 'p_name',
                 'p_number', 'p_bike', 'p_lat', 'p_lng','geometry', 'plz']]

# First explorations

## Total number of bookings (data points)

In [15]:
len(bremen)

1279878

## Total number of bikes - 443
- there are 443 different bikes presented in our data
- WESER-KURIER announced to provide 450 https://www.nextbike.de/de/news/start-fuer-wk-bike-in-bremen/

In [16]:
print(len(bremen.b_number.unique()))
print(bremen.b_number.unique())

443
[20668 20649 20964 20866 20799 20676 20855 20707 20815 20891 20721 20894
 20958 20942 20635 20838 20889 20827 20793 20797 20748 20791 20713 20790
 20810 20881 20817 20614 20768 20693 20626 20946 20915 20788 20684 20902
 20816 20704 20736 20912 20907 20769 20876 20943 20872 20924 20716 20968
 20761 20686 20804 20703 20967 20916 20756 20709 20725 20918 20903 20702
 20930 20688 20819 20690 20938 20843 20727 20874 20849 20651 20742 20751
 20658 20648 20904 20846 20697 20929 20669 20789 20859 20928 20854 20835
 20800 20771 20833 20937 20839 20935 20794 20715 20893 20621 20747 20900
 20667 20957 20712 20625 20657 20856 20888 20617 20766 20864 20724 20645
 20851 20805 20939 20921 20844 20731 20722 20934 20837 20719 20842 20899
 20734 20698 20616 20682 20743 20802 20735 20917 20858 20887 20798 20673
 20678 20932 20706 20896 20677 20806 20730 20732 20773 20630 20739 20679
 20687 20784 20647 20822 20832 20825 20738 20650 20685 20962 20906 20803
 20696 20922 20913 20670 20961 20963 20848 2084

### For a specific bike (b_number = 20668) check trips
- there seem to be many duplicates

In [17]:
bremen[bremen.b_number == 20668].sort_values('datetime')[:20]

,datetime,b_number,p_spot,p_place_type,trip,p_uid,p_bikes,b_bike_type,p_name,p_number,p_bike,p_lat,p_lng,geometry,plz
2241283,2019-01-20 00:00:00,20668,False,12,first,12097754,1,71,BIKE 20668,0.0,True,53.081698,8.812411,POINT (8.81241 53.08170),28195
2913437,2019-01-20 00:00:00,20668,False,12,first,12097754,1,71,BIKE 20668,0.0,True,53.081698,8.812411,POINT (8.81241 53.08170),28195
2241284,2019-01-20 01:33:00,20668,False,12,last,12097754,1,71,BIKE 20668,0.0,True,53.081767,8.812408,POINT (8.81241 53.08177),28195
2913438,2019-01-20 01:33:00,20668,False,12,last,12097754,1,71,BIKE 20668,0.0,True,53.081767,8.812408,POINT (8.81241 53.08177),28195
2242946,2019-01-22 09:15:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472,POINT (8.81147 53.08317),28195
2915100,2019-01-22 09:15:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472,POINT (8.81147 53.08317),28195
2242947,2019-01-22 09:42:00,20668,True,0,start,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472,POINT (8.81147 53.08317),28195
2915101,2019-01-22 09:42:00,20668,True,0,start,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472,POINT (8.81147 53.08317),28195
2242948,2019-01-22 09:49:00,20668,False,12,end,12155136,1,71,BIKE 20668,0.0,True,53.080356,8.795718,POINT (8.79572 53.08036),28195
2915102,2019-01-22 09:49:00,20668,False,12,end,12155136,1,71,BIKE 20668,0.0,True,53.080356,8.795718,POINT (8.79572 53.08036),28195


## Check for duplicates

In [18]:
# Checking for duplicates regarding the timestamp and bike_number 
bremen.duplicated(subset=['datetime', 'b_number']).sum()

639939

## Drop duplicates - half of data (there is perfectly one duplicate per data point)
- has to be done in preprocessor

In [19]:
bremen = bremen[bremen.duplicated(subset=['datetime', 'b_number'], keep='first') == False]

In [20]:
len(bremen)

639939

## Specifications of p_bikes
- number of available bikes at position/station

In [21]:
bremen.p_bikes.unique()

array([ 1,  2,  3,  5,  4,  0,  7,  6,  8,  9, 11, 10, 12, 15, 14, 13, 16,
       17, 31, 30, 32, 28, 20, 25, 26, 18, 23, 21, 27, 22, 29, 19])

## Specifications of b_bike_type
- two different types of bikes

In [22]:
bremen.b_bike_type.unique()

array([71, 29])

## Specifications of p_bike
- if True in flex zone and not at station
- opposite of p_spot

In [23]:
bremen.p_bike.unique()


array([ True, False])

## Specifications of p_spot
- at fixed station area
- if True there is a p_name (name of fixed station)
- opposite of p_bike

In [ ]:
bremen.p_spot.unique()

## Total number of p_uid
- unique ids of start and end locations
- includes 76 out of 83 ids of fixed stations

In [24]:
bremen.p_uid.nunique()

156809

- 76 out of 83 fixed stations are present in our data

In [25]:
len(bremen[bremen.p_spot == True].p_uid.unique())

73

## Import station data

In [26]:
stations = json_normalize(pd.read_json('../data/external/station_information.json').iloc[0,2])
stations.head(3)

,station_id,name,short_name,lat,lon,region_id,capacity
0,2351602,WESER-KURIER | Pressehaus,2910,53.076722,8.803943,379,5.0
1,7872668,Am Dobben,2925,53.078194,8.823250,379,NaN
2,7872701,Am Speicher XI,2926,53.097556,8.770056,379,NaN


In [27]:
len(stations) # number of fixed stations

83

## Specifications of p_name
- name of start/end location
- partly contains weird names like coordinates

In [28]:
bremen.p_name.nunique()

5955

In [29]:
bremen[bremen.p_name.str[:4] != 'BIKE'].p_name.unique()

array(['WeserTower', 'Pusdorfer Marktplatz', 'GEWOBA | Ludwig-Beck 2a',
       ..., ' Faulenstraße 65', '53.082872, 8.812058',
       'recording_91000056'], dtype=object)

## Total number of p_number
- short ids for fixed stations
- 0 if not at fixed station but in flex zone

In [30]:
print(len(bremen.p_number.unique()))
print(bremen.p_number.unique())

76
[    0.  2971.  2959.  2988.  2927.  2910.  2933.  2972.  2912.  2941.
  2938.  2949.  2913.  2976.  2985.  2928. 29860.  2929.  2970.  2936.
  2986.  2946.  2952.  2963.  2967.  2975.  2966.  2947.  2958.  2940.
  2945.  2962.  2961.  2969.  2987.  2937.  2930.  2965.  2934.  2957.
  2950.  2939.  2942.  2911.  2951.  2955.  2925.  2990.  2932.  2931.
  2926.  2948.  2944.  2992.  2956.  2954.  2991.  2935.  2914.  2997.
  2977.  2964.  2974.  2960.  2973.  2968.  2989.  2915.  2916.  2917.
  2919.  2918.  2920.  2923.  2922.  2921.]


## Specifications of p_bikes
- number of available bikes at position/station

In [31]:
bremen.p_bikes.unique()

array([ 1,  2,  3,  5,  4,  0,  7,  6,  8,  9, 11, 10, 12, 15, 14, 13, 16,
       17, 31, 30, 32, 28, 20, 25, 26, 18, 23, 21, 27, 22, 29, 19])

In [32]:
bremen[bremen.p_spot == False].p_bikes.max() # can there be more than one bike if not in flex zone - yes

2

## Specifications of trip
- [start, end, first, last]  (do not refer to duplicates)
- more specific analysis of trip attribute in the creation process of trips down below

In [33]:
bremen.trip.unique()

array(['first', 'last', 'start', 'end'], dtype=object)

In [34]:
len(bremen[(bremen.trip != 'first') & (bremen.trip != 'last')])

454796

In [35]:
bremen['datetime'] = pd.to_datetime(bremen['datetime']) # parse timestamp to datetime

### Sort data frame by timestamp
- has to be done in preprocessor

In [36]:
bremen = bremen.sort_values('datetime')

In [37]:
bremen[bremen.b_number == 20668][:20]

,datetime,b_number,p_spot,p_place_type,trip,p_uid,p_bikes,b_bike_type,p_name,p_number,p_bike,p_lat,p_lng,geometry,plz
2241283,2019-01-20 00:00:00,20668,False,12,first,12097754,1,71,BIKE 20668,0.0,True,53.081698,8.812411,POINT (8.81241 53.08170),28195
2241284,2019-01-20 01:33:00,20668,False,12,last,12097754,1,71,BIKE 20668,0.0,True,53.081767,8.812408,POINT (8.81241 53.08177),28195
2242946,2019-01-22 09:15:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472,POINT (8.81147 53.08317),28195
2242947,2019-01-22 09:42:00,20668,True,0,start,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472,POINT (8.81147 53.08317),28195
2242948,2019-01-22 09:49:00,20668,False,12,end,12155136,1,71,BIKE 20668,0.0,True,53.080356,8.795718,POINT (8.79572 53.08036),28195
2242949,2019-01-22 14:52:00,20668,False,12,start,12155136,1,71,BIKE 20668,0.0,True,53.080453,8.795549,POINT (8.79555 53.08045),28195
2242950,2019-01-22 15:35:00,20668,False,12,end,12162204,1,71,BIKE 20668,0.0,True,53.062169,8.875182,POINT (8.87518 53.06217),28207
2242951,2019-01-22 23:59:00,20668,False,12,last,12162204,1,71,BIKE 20668,0.0,True,53.061929,8.875118,POINT (8.87512 53.06193),28207
2243889,2019-01-23 00:00:00,20668,False,12,first,12162204,1,71,BIKE 20668,0.0,True,53.061929,8.875118,POINT (8.87512 53.06193),28207
2243890,2019-01-23 10:42:00,20668,False,12,last,12162204,1,71,BIKE 20668,0.0,True,53.062032,8.875321,POINT (8.87532 53.06203),28207


In [38]:
bremen[(bremen.b_number == 20668) & ((bremen.trip == 'first') & (bremen.datetime.dt.hour != 0))]

,datetime,b_number,p_spot,p_place_type,trip,p_uid,p_bikes,b_bike_type,p_name,p_number,p_bike,p_lat,p_lng,geometry,plz
2242946,2019-01-22 09:15:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472,POINT (8.81147 53.08317),28195
2246583,2019-01-25 07:46:00,20668,True,0,first,7873007,4,71,Domsheide,2933.0,False,53.074592,8.810182,POINT (8.81018 53.07459),28195
2264984,2019-02-11 13:24:00,20668,True,0,first,7873893,2,71,Woltmershauser Str.,2966.0,False,53.078528,8.779167,POINT (8.77917 53.07853),28197
2311718,2019-02-28 14:19:00,20668,False,12,first,13051288,1,71,BIKE 20668,0.0,True,53.081902,8.810073,POINT (8.81007 53.08190),28195
2313106,2019-03-01 16:45:00,20668,False,12,first,13075883,1,71,BIKE 20668,0.0,True,53.070702,8.830958,POINT (8.83096 53.07070),28203
2348561,2019-03-19 09:28:00,20668,False,12,first,13428348,1,71,BIKE 20668,0.0,True,53.078944,8.848838,POINT (8.84884 53.07894),28211
2356371,2019-03-25 10:56:00,20668,True,0,first,8638942,4,71,Föhrenstraße,2972.0,False,53.064904,8.879091,POINT (8.87909 53.06490),28207
2369451,2019-04-01 06:33:00,20668,False,12,first,13917223,1,71,Hamburger Straße 248,0.0,True,53.068182,8.849903,POINT (8.84990 53.06818),28205
2441464,2019-04-30 08:29:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472,POINT (8.81147 53.08317),28195
2445210,2019-05-01 03:51:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472,POINT (8.81147 53.08317),28195


In [44]:
# Visualizes number of trips clustered by locations

def plotDataPoints(latitudes, longitudes, zoom):
    
    # Create map centered at Bremens center
    m = create_bremen_map(zoom=zoom)
    
    marker_cluster = MarkerCluster().add_to(m)
        
    if len(latitudes) != len(longitudes):
        print('latitudes & longitudes must be the same size!')
    
    for i in range(len(latitudes)):
        
        lat = latitudes.iloc[i]
        lng = longitudes.iloc[i]
        
        folium.Circle(
            location=[lat,lng],
            radius=15,
            fill=True
        ).add_to(marker_cluster)
                  
    return m

## Visualization of 40000 randomly choosen data point locations

In [40]:
sample_points = bremen.sample(40000)

In [45]:
plotDataPoints(sample_points['p_lat'], sample_points['p_lng'], zoom=11).save('../reports/figures/samplevis_bremen_clustered.html')